Notebook to join greenness file to compiled ALEB lakes

Copied from Kurek_lakes_index, but This is just the joint part and then output is saved. See aleb_lakes_index.ipynb for continuation.

In [48]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
from matplotlib import pyplot as plt
import os
import numpy as np
from shapely.geometry import box

from land_cover.load import loadKurek, plot_dir, loadGreenness, kurek_bounds, loadLandCoverJoined, loadLandCoverJoinedShp, aleb_landcover_greenness_spatial
from land_cover.distance import calcMeanBoundDist

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Load merged dataset
gdf_land = loadLandCoverJoinedShp()

In [18]:
gdf_land.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Lake                       465 non-null    int64   
 1   Lat_DD                     465 non-null    float64 
 2   Lon_DD                     465 non-null    float64 
 3   AvgOfTempC                 308 non-null    float64 
 4   AvgOfpH                    308 non-null    float64 
 5   AvgOfALKum                 308 non-null    float64 
 6   AvgOfpCO2                  465 non-null    float64 
 7   StDevOfpCO                 308 non-null    float64 
 8   Name                       465 non-null    object  
 9   Reference                  165 non-null    object  
 10  mean_bound                 465 non-null    float64 
 11  max_bound_                 465 non-null    float64 
 12  Area_m2                    465 non-null    float64 
 13  Perim_m2                   

Add greenness

In [26]:
gdf_green = loadGreenness() #bounds='kurek')
len(gdf_green)

542997

In [33]:
# Crop for faster processing, project to common coordinate system
gdf_green = (
    gpd.sjoin(
        gdf_green,
        gpd.GeoDataFrame(geometry=[box(*gdf_land.to_crs(gdf_green.crs).total_bounds)], crs=gdf_green.crs),
        predicate="intersects",
    )
    .drop(columns="index_right")
    .to_crs(gdf_land.crs)
)
len(gdf_green)

60635

In [ ]:
# Join greenness to landcover geoms, Noting that some lakes will not have any greenness data because they are too small
len_gdf0 = len(gdf_land)
print(f"Original length of landcover dataset: {len_gdf0}")
gdf = gpd.sjoin(gdf_land[["geometry", "Lake"]], gdf_green, "left")

Original length of landcover dataset: 465


TODO: generalize these next two cells as a function

In [41]:
# Some lakes have multiple matches in GLAKES Due to narrow connections being split into multiple lakes. Need to aggregate by averaging.
agg_dict = {
    col: "mean" if pd.api.types.is_dtype_equal(gdf[col].dtype, "float64") else "first"
    for col in gdf.columns
}
gdf = gdf.groupby("Lake", as_index=False).agg(agg_dict)

In [ ]:
# Join greenness to remaining landcover fields
len_gdf0 = len(gdf)
gdf = pd.merge(
    gdf, gdf_land.drop(columns=["geometry"]), "inner", "Lake", validate="one_to_one"
).set_geometry("geometry")
gdf.crs = gdf_land.crs
print(f"After join: length of landcover dataset: {len(gdf)}")

After join: length of landcover dataset: 465


Note that only 324 / 465 have Liu greenness, and only 219 have Kuhn greenness

In [45]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 59 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   geometry                   465 non-null    geometry
 1   Lake                       465 non-null    int64   
 2   index_right                324 non-null    float64 
 3   OBJECTID                   324 non-null    float64 
 4   Lake_id                    324 non-null    float64 
 5   Area_bound                 324 non-null    float64 
 6   Area_PW                    324 non-null    float64 
 7   Continent                  324 non-null    object  
 8   Lat                        324 non-null    float64 
 9   Lon                        324 non-null    float64 
 10  GFed_flag                  324 non-null    float64 
 11  PFed_flag                  324 non-null    float64 
 12  Endo_flag                  324 non-null    float64 
 13  Rser_flag                  

Save

In [54]:
gdf.to_file(aleb_landcover_greenness_spatial)